## Import and data urls
We will load in three tables; title_basics, titles_akas, and title_ratings

In [1]:
import pandas as pd
import numpy as np

url1 = "https://datasets.imdbws.com/title.basics.tsv.gz"
url2 = "https://datasets.imdbws.com/title.akas.tsv.gz"
url3 = "https://datasets.imdbws.com/title.ratings.tsv.gz"

## Clean akas dataframe

In [2]:
%%time

# Load akas Data
# Because our data is seperated by a tab we indicate set='\t'.
# We will also use low_memory=False because the tables are very large
akas = pd.read_csv(url2, sep='\t', low_memory=False)

CPU times: total: 3min 8s
Wall time: 5min 9s


In [3]:
%%time

# Convert \\n to null (np.nan)
akas.replace({'\\N':np.nan}, inplace=True)

# Keep only the US movies
akas = akas[(akas['region'] == 'US')]
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365387 entries, 5 to 33752672
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1365387 non-null  object
 1   ordering         1365387 non-null  int64 
 2   title            1365387 non-null  object
 3   region           1365387 non-null  object
 4   language         3716 non-null     object
 5   types            965751 non-null   object
 6   attributes       45272 non-null    object
 7   isOriginalTitle  1364012 non-null  object
dtypes: int64(1), object(7)
memory usage: 93.8+ MB
CPU times: total: 2min 11s
Wall time: 3min 7s


## Clean title.basics dataframe

In [4]:
%%time

# Load basics Data
basics = pd.read_csv(url1, sep='\t', low_memory=False)

CPU times: total: 57.9 s
Wall time: 1min 16s


In [5]:
# Replace '\\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [6]:
# Drop any rows that have null values in either genre or runtimeMinutes.
basics = basics.dropna(subset=['runtimeMinutes', 'genres'])

In [7]:
# Keep only titleType==Movie
is_Movie = basics['titleType'] == 'movie'
basics = basics[is_Movie]

In [8]:
# Drop all rows where the start year is not between 2000-2022.

# We first need to change the column type to a numeric type.
print("\nData Type before conversion : ", basics['startYear'].dtype)
basics['startYear'] = pd.to_numeric(basics['startYear'])
print("\nData Type after conversion : ", basics['startYear'].dtype)

#Now we can make and apply our filters
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]


Data Type before conversion :  object

Data Type after conversion :  float64


In [9]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [10]:
# Filter the basics table down to only include the US movies by using the filtered akas dataframe above
keepers_basics = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers_basics]

basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83336 entries, 34793 to 9354438
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          83336 non-null  object 
 1   titleType       83336 non-null  object 
 2   primaryTitle    83336 non-null  object 
 3   originalTitle   83336 non-null  object 
 4   isAdult         83336 non-null  object 
 5   startYear       83336 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  83336 non-null  object 
 8   genres          83336 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.4+ MB


## Clean ratings dataframe

In [11]:
%%time

# Load ratings Data
ratings = pd.read_csv(url3, sep='\t', low_memory=False)

CPU times: total: 2.14 s
Wall time: 3.08 s


In [12]:
# Replace \\n with null
ratings.replace({'\\N':np.nan}, inplace=True)

# Filter the ratings table down to only include the US movies by using the filtered akas dataframe
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 473951 entries, 0 to 1245157
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         473951 non-null  object 
 1   averageRating  473951 non-null  float64
 2   numVotes       473951 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.5+ MB


## save cleaned files in Data folder

In [13]:
# Save current dataframes to files.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
print("Done!")

Done!
